# 네이버 영화평 감성 분석

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
# '네이버 영화 리뷰 데이터'로 검색

train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [3]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

### 1. 데이터 전처리

- 트레인 데이터 셋

In [4]:
# 중복 여부 확인
train_df.document.nunique()

146182

In [5]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146183, 3)

In [6]:
# Null 데이터가 있는지 확인
train_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [7]:
# Null 데이터 제거
train_df = train_df.dropna(how='any')
train_df.shape

(146182, 3)

In [8]:
# 긍정, 부정 레이블의 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- 테스트 데이터 셋에도 적용

In [9]:
# 중복 여부 확인
test_df.document.nunique()

49157

In [10]:
# 중복 데이터 제거
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49158, 3)

In [11]:
# Null 데이터가 있는지 확인
test_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [12]:
# Null 데이터 제거
test_df = test_df.dropna(how='any')
test_df.shape

(49157, 3)

In [13]:
# 긍정, 부정 레이블의 분포
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

### 2. 텍스트 전처리

In [14]:
import warnings
warnings.filterwarnings(action='ignore')

- train dataset

In [15]:
# 한글과 공백 이외는 제거
train_df['document'] = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [16]:
# '' 데이터는 Nan으로 변환한 후 제거
train_df['document'].replace('', np.nan, inplace=True)
train_df.document.isnull().sum()

391

In [17]:
train_df = train_df.dropna(how='any')
train_df.shape

(145791, 3)

- test dataset

In [18]:
test_df['document'] = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
test_df['document'].replace('', np.nan, inplace=True)
test_df = test_df.dropna(how='any')
test_df.shape

(48995, 3)

In [19]:
train_df.to_csv('naver_movie_train.tsv', sep='\t', index=False)
test_df.to_csv('naver_movie_test.tsv', sep='\t', index=False)

### 3. 한글 처리

In [20]:
from konlpy.tag import Okt
okt = Okt()

In [21]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','ㅋㅋ','ㅠㅠ','ㅎㅎ']

In [22]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
okt.morphs(text)

['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정']

In [23]:
okt.morphs(text, stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [24]:
from tqdm import tqdm_notebook
X_train = []
for sentence in tqdm_notebook(train_df.document):
    morphs = okt.morphs(sentence, stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stopwords])
    X_train.append(temp_X)

  0%|          | 0/145791 [00:00<?, ?it/s]

In [25]:
X_test = []
for sentence in tqdm_notebook(test_df.document):
    morphs = okt.morphs(sentence, stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stopwords])
    X_test.append(temp_X)

  0%|          | 0/48995 [00:00<?, ?it/s]

In [26]:
y_train = train_df.label.values
y_test = test_df.label.values

### 4. Feature 변환, 모델 학습/예측/평가

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [28]:
cvect = CountVectorizer()
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)

In [29]:
lr = LogisticRegression()
lr.fit(X_train_cv, y_train)

LogisticRegression()

In [30]:
pred = lr.predict(X_test_cv)
accuracy_score(y_test, pred)

0.8253699357077253

### 5. 실제 테스트

In [31]:
review1 = '아름다운 음악과 아름다운 풍광~ 그렇지 못한 현실이 찡하네요~'
review2 = '메시지와 작위성의 불협화음!!!'

In [32]:
review1 = review1.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')
review1

'아름다운 음악과 아름다운 풍광~ 그렇지 못한 현실이 찡하네요~'

In [33]:
import re
review1 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review1)
review1

'아름다운 음악과 아름다운 풍광 그렇지 못한 현실이 찡하네요'

In [34]:
morphs = okt.morphs(review1)
review = ' '.join([word for word in morphs if not word in stopwords])
review

'아름다운 음악 아름다운 풍 광 그렇지 못 현실 찡하네요'

In [35]:
review_cv = cvect.transform([review])
pred = lr.predict(review_cv)
pred[0]

1

In [36]:
review2 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review2)
morphs = okt.morphs(review2)
review = ' '.join([word for word in morphs if not word in stopwords])
review_cv = cvect.transform([review])
pred = lr.predict(review_cv)
pred[0]

0

In [37]:
reviews = ['아름다운 음악과 아름다운 풍광~ 그렇지 못한 현실이 찡하네요~',
           '메시지와 작위성의 불협화음!!!']

In [38]:
review_list = []
for review in reviews:
    review = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review)
    morphs = okt.morphs(review, stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stopwords])
    review_list.append(temp_X)

In [39]:
review_cv = cvect.transform(review_list)
pred = lr.predict(review_cv)
pred

array([1, 0], dtype=int64)

### 6. GridSearchCV로 최적 파라미터 찾기

In [40]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [41]:
pipeline = Pipeline([
    ('cvect', CountVectorizer()),
    ('lr', LogisticRegression())
])
params = {
    'cvect__ngram_range': [(1,1),(1,2)],
    'cvect__max_df': [0.9, 0.99],
    'cvect__min_df': [1, 3],
    'lr__C': [1, 5]
}

In [42]:
grid_pipe = GridSearchCV(
    pipeline, param_grid=params, cv=3, scoring='accuracy', n_jobs=-1
)
%time grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_score_, grid_pipe.best_params_)

Wall time: 2min 36s
0.840538853564349 {'cvect__max_df': 0.9, 'cvect__min_df': 1, 'cvect__ngram_range': (1, 2), 'lr__C': 1}


In [43]:
pred = grid_pipe.best_estimator_.predict(X_test)
acc = accuracy_score(y_test, pred)
print(f'CounterVectorizer + LogisticRegression 정확도: {acc:.4f}')

CounterVectorizer + LogisticRegression 정확도: 0.8462
